In [ ]:
# 导入需要用到的模块
## 数值计算模块numpy。
import numpy as np
## 数据处理模块pandas。
import pandas as pd
## 字符串匹配模块re，很多功能是基于正则表达式实现的。
import re

In [ ]:
# 读取数据
## 从工作目录下的'data.xls'文件中的'data'表中读取数据，没有索引列，空值用'NA'填充。请确认文件路径、文件名、表名正确。
data = pd.read_excel('data.xls', 'data', index_col=None, na_values=['NA'])
## 输出读取数据的数量
print('数据量为： ' + str(len(data)) + ' 行。')

In [ ]:
# 数据预处理

## 数据筛选，删除不需要的数据行。处理了“审核状态”、“病例分类”、“病例分类2”三列。
### 删除“审核状态”列值为“已删除卡”的数据行
data = data[data['审核状态'] != '已删除卡']
### 删除“病例分类”列值为“疑似病例”的数据行
data = data[data['病例分类'] != '疑似病例']
### 删除“病例分类2”列值为“慢性”的数据行
data = data[data['病例分类2'] != '慢性']

### 校验数据筛选操作，查看此三列的值域是否还有异常值。
print(data['审核状态'].value_counts())
print(data['病例分类'].value_counts())
print(data['病例分类2'].value_counts())

### 输出数据筛选后的数据量
print('数据筛选后，数据量为： ' + str(len(data)) + ' 行。')

In [ ]:
## 读取辅助表“疾病”、“年龄”、“街镇”三个表，用于生成新列。“辅助表.xls”文件中，包含了表“疾病”、“年龄”、“街镇”、“年龄报表”、“职业报表”、“月报1”、“月报2”等相关元数据。请确保元数据可用，比如，记录唯一性等。
jibing = pd.read_excel('辅助表.xls', '疾病', index_col=None, na_values=['NA'])
age = pd.read_excel('辅助表.xls', '年龄', index_col=None, na_values=['NA'])
jiezhen = pd.read_excel('辅助表.xls', '街镇', index_col=None, na_values=['NA'])

In [ ]:
## 对“疾病名称”列进行整理,生成新列“疾病名称1”。规范“其它”相关诊断和“肺结核”相关诊断。
#### “疾病名称”列值匹配“其它疾病”、“其它：”、“其它:”时，“疾病名称1”值为“其它”。
data['疾病名称1'] = data['疾病名称'].apply(lambda x: '其它' if (re.match('其它疾病',x) or re.match('其它:',x) or re.match('其它：',x)) else x)
#### “疾病名称1”列值匹配“涂（+）”和“仅培阳”时，“疾病名称1”值为“病原学阳性”。
data['疾病名称1'] = data['疾病名称1'].apply(lambda x: '病原学阳性' if (re.match('涂（\+）',x) or re.match('仅培阳',x)) else x)
#### “疾病名称1”列值匹配“菌（-）”时，“疾病名称1”值为“病原学阴性”。
data['疾病名称1'] = data['疾病名称1'].apply(lambda x: '病原学阴性' if (re.match('菌（-）',x)) else x)
#### “疾病名称1”列值匹配“未痰检”和“结核性胸膜炎”时，“疾病名称1”值为“无病原学结果”。
data['疾病名称1'] = data['疾病名称1'].apply(lambda x: '无病原学结果' if (re.match('未痰检',x) or re.match('结核性胸膜炎',x)) else x)

## 结合辅助表，根据“疾病名称1”列，生成新列'疾病名称2'、'疾病分组1'、'疾病分组2'、'疾病分组3'。一个疾病，有很多其它属性，比如“甲乙丙”分类、传播途径等，这里予以添加、完善。
data = pd.merge(data, jibing.loc[:, ['疾病名称1', '疾病名称2', '疾病分组1', '疾病分组2', '疾病分组3']], how = 'left', on = '疾病名称1')

## 结合辅助表，根据“年龄”列，生成新列“年龄分组”。
data = pd.merge(data, age.loc[:, ['年龄', '年龄分组']], how = 'left', on = '年龄')

## 结合辅助表，根据“现住地址国标”列，生成新列“街镇分组”。
data = pd.merge(data, jiezhen.loc[:, ['现住地址国标', '街镇分组']], how = 'left', on = '现住地址国标')

## 根据“病人属于”列，生成新列“户籍”。选取“病人属于”列为'其他省'、'外籍'和'港澳台'的'户籍'列，予以赋值。
data.loc[data['病人属于'].isin(['其他省','外籍','港澳台']),'户籍'] = '外来'
data.loc[data['病人属于'].isin(['本县区','本市其它县区','本省其它地市']),'户籍'] = '本市'

## 根据“人群分类”列，生成新列“人群分类2”。“人群分类”列值匹配“其它：”、“其它:”时，“人群分类2”值为“其它”。
data['人群分类2'] = data['人群分类'].apply(lambda x: '其它' if re.match('其它',x) else x)

In [ ]:
## 时间属性整理

### 根据“订正终审时间”列，生成新列“年-终审”。“年-终审”列是以周为单位划分年份。比如，2020年第1周始于2019年12月30日，2020年第53周终于2021年01月03日。选取“订正终审时间”在某个时间范围内的“年-终审”列，进行赋值操作。
data.loc[(data['订正终审时间'] >= pd.Timestamp('2021-01-04')) & (data['订正终审时间'] < pd.Timestamp('2022-01-03')),'年-终审'] = '2021'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2019-12-30')) & (data['订正终审时间'] < pd.Timestamp('2021-01-04')),'年-终审'] = '2020'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2018-12-31')) & (data['订正终审时间'] < pd.Timestamp('2019-12-30')),'年-终审'] = '2019'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2018-01-01')) & (data['订正终审时间'] < pd.Timestamp('2018-12-31')),'年-终审'] = '2018'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2017-01-02')) & (data['订正终审时间'] < pd.Timestamp('2018-01-01')),'年-终审'] = '2017'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2016-01-04')) & (data['订正终审时间'] < pd.Timestamp('2017-01-02')),'年-终审'] = '2016'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2014-12-29')) & (data['订正终审时间'] < pd.Timestamp('2016-01-04')),'年-终审'] = '2015'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2013-12-30')) & (data['订正终审时间'] < pd.Timestamp('2014-12-29')),'年-终审'] = '2014'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2012-12-31')) & (data['订正终审时间'] < pd.Timestamp('2013-12-30')),'年-终审'] = '2013'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2012-01-02')) & (data['订正终审时间'] < pd.Timestamp('2012-12-31')),'年-终审'] = '2012'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2011-01-03')) & (data['订正终审时间'] < pd.Timestamp('2012-01-02')),'年-终审'] = '2011'
data.loc[(data['订正终审时间'] >= pd.Timestamp('2010-01-04')) & (data['订正终审时间'] < pd.Timestamp('2011-01-03')),'年-终审'] = '2010'
### 根据“订正终审时间”列，生成新列“月-终审”。取得“订正终审时间”的月值，赋值给“月-终审”列。
data['月-终审'] = pd.DatetimeIndex(data['订正终审时间']).month
### 根据“订正终审时间”列，生成新列“周-终审”。取得“订正终审时间”的周值，赋值给“周-终审”列。
data['周-终审'] = pd.DatetimeIndex(data['订正终审时间']).weekofyear

In [ ]:
## 对数据整理中生成的新列，进行校验，查看是否全部赋值。如果有空值，需要进一步完善数据整理。计算某列值为空的数据的长度，予以输出。
print('疾病分类分组-疾病名称1空值数：  '+str(len(data[data['疾病名称1'].isna()])))
print('疾病分类分组-疾病名称2空值数：  '+str(len(data[data['疾病名称2'].isna()])))
print('疾病分类分组-疾病分组1空值数：  '+str(len(data[data['疾病分组1'].isna()])))
print('疾病分类分组-疾病分组2空值数：  '+str(len(data[data['疾病分组2'].isna()])))
print('疾病分类分组-疾病分组3空值数：  '+str(len(data[data['疾病分组3'].isna()])))
print('年龄分组空值数：                '+str(len(data[data['年龄分组'].isna()])))
print('街镇分组空值数：                '+str(len(data[data['街镇分组'].isna()])))
print('年-终审空值数：                 '+str(len(data[data['年-终审'].isna()])))
print('月-终审空值数：                 '+str(len(data[data['月-终审'].isna()])))
print('周-终审空值数：                 '+str(len(data[data['周-终审'].isna()])))
print('人群分类空值数：                '+str(len(data[data['人群分类2'].isna()])))

In [ ]:
## 完成数据预处理，保存。
data.to_excel('data-done.xls', sheet_name='data')